In [5]:
import time
time_start = time.time()
import re
import os
import numpy as np
import scipy as sp
import mne
import nilearn.plotting as nplt
import nilearn.image as image
import pickle
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from sklearn.metrics import roc_auc_score
from nodestimation.node_estimate import Node
from nodestimation.timewindow import TimeWindow, sliding_window
import nibabel as nib
import matplotlib.pyplot as plt
import nodestimation.project.path as path
from nodestimation.project.subject import Subject
from nodestimation.pipeline import pipeline

from nodestimation.project.structures import subject_data_types
subjects = pipeline(con_method=['coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli'],
                    freq_diaps=[(0.5, 4), (4, 7), (7, 14), (14, 30)])
print(subjects[0].data.keys())
print(len(subjects[0].nodes))
print(subjects[0].nodes[1].type)

All computation has been already done, loading of the existing file with the solution...
dict_keys(['raw', 'raw_fp', 'bem', 'src', 'trans', 'fwd', 'eve', 'epo', 'cov', 'ave', 'inv', 'stc', 'resec', 'resec_mni', 'parc', 'coords', 'con', 'psd', 'nodes'])
68
spared


In [ ]:
fig, ax = plt.subplots()
ax.plot(psd['0.5-4Hz']['bankssts-lh']['freq'], psd['0.5-4Hz']['bankssts-lh']['psd'].mean(axis=0))
# ax.set_xlabel('Freq (Hz)')
# ax.set_xlim(psd['0.5-4Hz']['bankssts-lh']['freq'][[0, -1]])
# ax.set_ylabel('Power Spectral Density')

In [ ]:
print(connectivity['0.5-4Hz']['plv']['con'].shape)
print(connectivity['0.5-4Hz']['pli']['con'].shape)
print(connectivity['0.5-4Hz']['ppc']['con'].shape)
print(connectivity['0.5-4Hz']['coh']['con'].shape)
print(connectivity['0.5-4Hz']['imcoh']['con'].shape)
print(psd['0.5-4Hz']['bankssts-lh']['freq'].shape)

In [ ]:
con = connectivity['0.5-4Hz']['imcoh-based']['con'][:, :, 0]

con_s = np.zeros(con.shape)
for i in range(con.shape[0]):
    for j in range(con.shape[1]):
        con_s[j, i] = con[i, j]
# print('con')
# for i in range(con.shape[0]):
#     print(con[i, 0:10])
# print('con_s')
# for i in range(con.shape[0]):
#     print(con_s[i, 0:10])

plt.imshow(con)
plt.show()
plt.imshow(con.T)
plt.show()
plt.imshow(con + con_s)
plt.show()

In [ ]:
label_names = [label.name for label in labels]
lh_labels = [name for name in label_names if name.endswith('lh')]
rh_labels = [name for name in label_names if name.endswith('rh')]

label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])

In [ ]:
print(connectivity.keys())
print(connectivity['0.5-4Hz'].keys())
print(connectivity['0.5-4Hz']['coh-based'].keys())

In [ ]:
# for freq_band in connectivity:
#     for method in connectivity[freq_band]:
#         fig = plt.figure(num=None, figsize=(8, 8), facecolor='black')
#         mne.viz.plot_connectivity_circle(connectivity[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
#                      node_angles=node_angles, node_colors=node_colors,
#                      title='All-to-All Connectivity Epilepsy '
#                      'Condition ({} for {})'.format(method, freq_band), fig=fig)

methods = [
    'coh-based',
    'imcoh-based',
    'plv-based',
    'ppc-based',
    'pli-based'
]

freq_bands = [
    '0.5-4Hz',
    '4-7Hz',
    '7-14Hz',
    '14-30Hz'
]

for method in methods:
    for freq_band in freq_bands:
        fig = plt.figure(num=None, figsize=(25, 25), facecolor='black')
        mne.viz.plot_connectivity_circle(connectivity[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
                                         node_angles=node_angles, node_colors=node_colors,
                                         title='All-to-All Connectivity Epilepsy Condition ({} for {})'
                                         .format(method, freq_band), padding=8, fontsize_title=35, fontsize_colorbar=25,
                                         fontsize_names=20, fig=fig
                                         )

In [ ]:
del stc, src, raw, fwd,\
    bem, labels, label_ts, inv, cov
